In [1]:
import time
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service as ChromeService
from bs4 import BeautifulSoup
import requests

In [2]:
data_dir = '../data/'

In [3]:
from fake_useragent import UserAgent
ua = UserAgent()

In [4]:
# configure webdriver
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [5]:
#Initialize chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, chrome_options=options)

driver.get("https://www.metacritic.com/search/game/Grand%20theft%20auto/results?search_type=advanced&page=0")

# Get list of product urls
url_list = []

while True:
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    games = soup.find_all(class_='product_title basic_stat')
    
    for game in games:
        url_list.append('https://www.metacritic.com'+game.find('a', href=True)['href']+"/critic-reviews")
    
    try:
        next_button = driver.find_element_by_link_text('Next')
        next_button.click()
    except:
        driver.close()
        break



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\siraj\.wdm\drivers\chromedriver\win32\98.0.4758.80\chromedriver.exe] found in cache
<ipython-input-5-363cd6d98c62>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, chrome_options=options)


In [16]:
#Initialize data structures
critic_reviews = {'game_id':[], 'game_name': [], 'score':[], 'author':[], 'date':[], 'summary': []}

game_id = 0

# Parse review data
for game in url_list:
    
    try:
        page = requests.get(game, headers={'User-Agent': ua.chrome})
        soup = BeautifulSoup(page.text, 'html.parser')

        game_name = soup.find(class_ = 'product_title').find('h1').text.lstrip().replace('\n', '')

        reviews = soup.find(class_= 'body product_reviews')
        score_elems = reviews.find_all(class_="review_grade")
        author_elems = reviews.find_all(class_="source")
        date_elems = reviews.find_all(class_="date")
        sum_elems = reviews.find_all(class_="review_body")
        
        if len(date_elems)==len(sum_elems):

            for elem in score_elems:
                critic_reviews['game_id'].append(game_id)
                critic_reviews['game_name'].append(game_name)
                critic_reviews['score'].append(int(elem.text))

            for elem in author_elems:
                critic_reviews['author'].append(elem.text)

            for elem in date_elems:
                critic_reviews['date'].append(elem.text)

            for elem in sum_elems:
                critic_reviews['summary'].append(elem.text.lstrip().rstrip())

            game_id+=1
        
    except:
        pass

In [18]:
import pandas as pd

reviews = pd.DataFrame(critic_reviews)
reviews.to_csv(data_dir+'gta_reviews.csv')

In [19]:
reviews

,game_id,game_name,score,author,date,summary
0,0,Grand Theft Auto V,100,Playstation Official Magazine UK,"Jan 13, 2015",The best game on PS3 is now the best thing on ...
1,0,Grand Theft Auto V,100,LEVEL (Czech Republic),"Jan 5, 2015",An excellent remake of an outstanding game tha...
2,0,Grand Theft Auto V,100,Quarter to Three,"Dec 19, 2014","With this latest version, Rockstar’s latest ga..."
3,0,Grand Theft Auto V,100,Playstation Official Magazine Australia,"Dec 16, 2014",GTA V on PS4 does exactly what gamers everywhe...
4,0,Grand Theft Auto V,100,Toronto Sun,"Dec 2, 2014",Another rampage through Los Santos in the eyes...
...,...,...,...,...,...,...
402,58,Grand Theft Auto: The Trilogy - The Definitive...,40,Cubed3,"Nov 23, 2021","Good, fun games are good, fun games, and even ..."
403,58,Grand Theft Auto: The Trilogy - The Definitive...,40,Nintendo Life,"Nov 12, 2021",Grand Theft Auto: The Trilogy - The Definitive...
404,58,Grand Theft Auto: The Trilogy - The Definitive...,40,VGC,"Nov 12, 2021",Few games have had such a lasting impact on ho...
405,58,Grand Theft Auto: The Trilogy - The Definitive...,30,Nintendo Insider,"Jan 28, 2022","These are all great games, some of the most in..."
